In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [70]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [71]:
data_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [72]:
data_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [73]:
data_fake['class'] = 0
data_true['class']=1

In [74]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [75]:
# removing last 10 rows for manual testing
data_fake_manual_testing = data_fake.tail(10)
data_fake = data_fake.iloc[:-10]

data_true_manual_testing = data_true.tail(10)
data_true = data_true.iloc[:-10]

In [76]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [77]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\prana\AppData\Local\Temp\ipykernel_17872\277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\prana\AppData\Local\Temp\ipykernel_17872\277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [78]:
data_merge = pd.concat([data_fake,data_true])

In [79]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [80]:
data = data_merge.drop(['title','subject','date'], axis = 1)

In [81]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [82]:
data = data.sample(frac=1)

In [83]:
data.columns

Index(['text', 'class'], dtype='object')

In [84]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [85]:
data.columns

Index(['text', 'class'], dtype='object')

In [86]:
data.head()

,text,class
0,Maybe you ve finally heard by now that a mosqu...,0
1,Corruption is nothing new for Hillary and Bill...,0
2,WASHINGTON (Reuters) - President Donald Trump ...,1
3,BUENOS AIRES (Reuters) - Sounds detected by pr...,1
4,SEOUL/WASHINGTON (Reuters) - Two U.S. strategi...,1


In [87]:
import re
import string

def word(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove text within square brackets
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove non-word characters (replace with space)
    text = re.sub(r'\W+', ' ', text)
    
    # Remove URLs (both http/https and www)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove punctuation
    text = re.sub(rf'[{string.punctuation}]', '', text)
    
    # Remove newlines
    text = re.sub(r'\n', '', text)
    
    # Remove alphanumeric words
    text = re.sub(r'\w*\d\w*', '', text)
    
    return text


In [88]:
data['text'] = data['text'].apply(word)

In [89]:
X = data['text']
y= data['class']

In [90]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25)


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer()
xv_train = vector.fit_transform(X_train)
xv_test = vector.transform(X_test)

In [92]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [93]:
pred_lr = LR.predict(xv_test)

In [94]:
LR.score(xv_test,y_test)

0.9867201426024955

In [95]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5882
           1       0.98      0.99      0.99      5338

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [96]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)


DecisionTreeClassifier()

In [97]:
pred_dt = DT.predict(xv_test)

In [98]:
DT.score(xv_test,y_test)

0.9966131907308378

In [99]:
print(classification_report(y_test,pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5882
           1       1.00      1.00      1.00      5338

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [100]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier()
GB.fit(xv_train,y_train)

GradientBoostingClassifier()

In [101]:
pred_gb = GB.predict(xv_test)

In [102]:
GB.score(xv_test,y_test)

0.9959001782531194

In [103]:
print(classification_report(y_test,pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5882
           1       0.99      1.00      1.00      5338

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [104]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [105]:
pred_rf = RF.predict(xv_test)

In [106]:
RF.score(xv_test,y_test)

0.9874331550802139

In [107]:
print(classification_report(y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5882
           1       0.98      0.99      0.99      5338

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [109]:
def output_label(n):
    if n==0:
        return "Fake News"
    else:
        return "Real News"
    
def manual_testing(news):
    # Create a DataFrame with the news article
    testing_news = {'text': [news]}
    new_def_test = pd.DataFrame(testing_news)
    
    # Apply the text preprocessing function
    new_def_test['text'] = new_def_test['text'].apply(word)
    
    # Extract the processed text and transform using the vectorizer
    new_x_test = new_def_test['text']
    new_xv_test = vector.transform(new_x_test)
    
    # Predict using each model
    pred_LR = LR.predict(new_xv_test)[0]
    pred_DT = DT.predict(new_xv_test)[0]
    pred_GB = GB.predict(new_xv_test)[0]
    pred_RF = RF.predict(new_xv_test)[0]
    
    # Map numerical predictions to labels
    predictions = {
        'Logistic Regression': output_label(pred_LR),
        'Decision Tree': output_label(pred_DT),
        'Gradient Boosting': output_label(pred_GB),
        'Random Forest': output_label(pred_RF)
    }
    
    return predictions





In [117]:
# Example usage:
news_article = input("Enter the News Article Content")
predictions = manual_testing(news_article)
print(predictions)

{'Logistic Regression': 'Fake News', 'Decision Tree': 'Fake News', 'Gradient Boosting': 'Fake News', 'Random Forest': 'Fake News'}
